# Importing Modules

In [19]:
import fitz
from tqdm.auto import tqdm

# Formatting text

In [20]:
def text_format(text: str)->str :
    cleaner_text = text.replace("\n", " ").strip()

    return cleaner_text

In [21]:
def get_text_from_source(path: str)->list[dict]:
    doc = fitz.open(path)
    pages_text = []

    for pageno, pagecontent in tqdm(enumerate(doc)):
        text = pagecontent.get_text()
        text = text_format(text=text)
        pages_text.append({"Page No.": pageno, "page_char_count": len(text), "page_word_count": len(text.split(" ")), "page_sentence_count": len(text.split(".")), "page_token_count": len(text)/4, "text": text})
        
    return pages_text

# Using our custom function


In [23]:
import random

text_info = get_text_from_source(path="test.pdf")
random.sample(text_info, k=3)


48it [00:00, 752.64it/s]


[{'Page No.': 17,
  'page_char_count': 1584,
  'page_word_count': 257,
  'page_sentence_count': 28,
  'page_token_count': 396.0,
  'text': '5 Framework Overview and Comparison This section presents an overview of major GPU programming frameworks used in high- performance computing: CUDA, HIP with CUDA Backend, HIP on ROCm, and OpenCL. A comparison based on hardware compatibility, ease of use, and programming features is also provided. 5.1 CUDA (Compute Unified Device Architecture) CUDA is a parallel computing platform and programming model developed by NVIDIA. It allows developers to use C/C++ syntax to write programs that run directly on NVIDIA GPUs. 1. CUDA provides low-level control over thread hierarchy, memory management, and synchronization. 2. It enables fine-tuned performance on NVIDIA hardware. 5.2 HIP with CUDA Backend on NVIDIA GPUs HIP (Heterogeneous-Compute Interface for Portability) is a C++ runtime API developed by AMD that allows developers to write portable code target